In [1]:
import mpmath as mp

from code.python.mpmath_integration import quad_phi

In [2]:
x = mp.mpf('1/2')
alpha = mp.mpf('4')
beta = mp.mpf('3')
mu = mp.mpf('1/4')
delta = mp.mpf('1')

xmu = x - mu

gamma = mp.sqrt(alpha ** 2 - beta ** 2)
omega = mp.sqrt(xmu ** 2 + delta ** 2)

### 1. Expansion a -> 0

In [3]:
a = x-mu
b = -beta

In [4]:
def normcdf(x):
    return mp.erfc(-x / mp.sqrt(mp.mpf('2'))) / mp.mpf('2')

In [5]:
t = 1

normcdf(a / mp.sqrt(t) + b * mp.sqrt(t))

mpf('0.0029797632350545577')

In [6]:
normcdf(b * mp.sqrt(t))

mpf('0.0013498980316300957')

In [7]:
A = mp.exp(-b**2/2 * t) / mp.sqrt(2*mp.pi) / mp.sqrt(t)

In [8]:
s1 =  a
s2 = -a**2 * b / 2
s3 =  a**3 * (b**2 * t - 1) / t / 6
s4 = -a**4 * (b**3 * t - 3 * b) / t / 24
s5 =  a**5 * (b**4 * t**2 - 6 * b**2 * t + 3) / t**2 / 120
s6 = -a**6 * (b**5 * t**2 - 10 * b**3 * t + 15 * b) / t**2 / 720

S = s1 + s2 + s3 + s4 + s5 + s6

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.0029797690734740699')

In [9]:
s1, s2, s3, s4, s5, s6

(mpf('0.25'),
 mpf('0.09375'),
 mpf('0.020833333333333332'),
 mpf('0.0029296875'),
 mpf('0.000244140625'),
 mpf('6.1035156250000003e-6'))

In [10]:
print(
    1,
    b,
    (b**2 * t - 1) / t,
    (b**3 * t - 3 * b) / t,
    (b**4 * t**2 - 6 * b**2 * t + 3) / t**2,
    (b**5 * t**2 - 10 * b**3 * t + 15 * b) / t**2
)

1 -3.0 8.0 -18.0 30.0 -18.0


In [11]:
def P(k, t, b):
    s = 0
    for m in range(int(mp.floor(k/2)) + 1):
        s += (-1) ** m / mp.factorial(m) / mp.factorial(k - 2 * m) * b ** (k - 2*m) / 2 ** m / t ** m

    return mp.factorial(k) * s

In [22]:
k = 0
P(k, t, b)

mpf('1.0')

In [21]:
K = 10
S = 0
for k in range(1, K):
    S += (-1) ** (k + 1) * a ** k / mp.factorial(k) * P(k-1, t, b)

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.0029797632329101242')

Hermite probabilistic

In [23]:
def hermite_prob(k, z):
    s = 0
    for m in range(int(mp.floor(k/2)) + 1):
        s += (-1) ** m / mp.factorial(m) / mp.factorial(k - 2 * m) * z ** (k - 2*m) / 2 ** m

    return mp.factorial(k) * s    

In [24]:
k = 10

P(k, t, b)

mpf('9503.9999999999927')

In [25]:
hermite_prob(k, b * mp.sqrt(t)) / t ** (k / 2)

mpf('9503.9999999999927')

In [26]:
mp.hermite(k, b * mp.sqrt(t / 2)) / (2 * t) ** (k / 2)

mpf('9504.0000000000055')

In [27]:
K = 40
S = 0
for k in range(1, K):
    S += (-1) ** (k + 1) * a ** k / mp.factorial(k) * mp.hermite(k-1, b * mp.sqrt(t / 2)) / (2 * t) ** ((k-1) / 2)

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.0029797632350545581')

In [29]:
K = 40
S = 0
for k in range(K):
    S += (-1) ** k * a ** (k + 1) / mp.factorial(k + 1) * mp.hermite(k, b * mp.sqrt(t / 2)) / (2 * t) ** (k / 2)

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.0029797632350545581')

### 2. Expansion

In [31]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
mp_result

mpf('0.13323254044801654')

In [32]:
quad_phi(mu, alpha, beta, mu, delta, digits=100)

mpf('0.055220468434900599')

In [33]:
C = delta * mp.exp(delta * gamma) / (2 * mp.pi)

In [45]:
mp.mp.dps = 30

N = 30
s = 0
for k in range(N):
    r = (-1) ** k * xmu ** (k + 1) / mp.factorial(k + 1) / 2 ** (k / 2)
    q = mp.quad(lambda t: t ** (-k/2 - 2) * mp.hermite(k, b * mp.sqrt(t / 2)) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
    s += r * q

C * s

mpf('0.0780120720131159588118227127856339')

In [46]:
quad_phi(mu, alpha, beta, mu, delta, digits=100) + C * s

mpf('0.133232540448016558081602114032476')

Coefficients

Ik+1

In [57]:
k = 1
mp.quad(lambda t: t ** (-(k+1)/2 - 2) * mp.hermite(k+1, b * mp.sqrt(t / 2)) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

mpf('0.683932605879470771419335615140985')

In [52]:
mp.quad(lambda t: t ** (-(k+1)/2 - 2) * (2 * b * mp.sqrt(t / 2) * mp.hermite(k, b * mp.sqrt(t / 2)) -2*k *  mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

mpf('0.683932605879470771419335615140985')

Ik

In [55]:
Ik = mp.quad(lambda t: t ** (-k/2 - 2) * mp.hermite(k, b * mp.sqrt(t / 2)) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
Ik, Ik * b * mp.sqrt(2)

(mpf('-0.423704002373866957114707624096963'),
 mpf('1.79762383976665413173532191723627'))

Ik-1

In [58]:
-2*k * mp.quad(lambda t: t ** (-(k+1)/2 - 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

mpf('-1.11369123388718336031598630209519')

In [61]:
mp.quad(lambda t: t ** (-(k+1)/2 - 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

mpf('0.556845616943591680157993151047595')

In [ ]:
z = alpha ** 2 / 2
r = delta / alpha

In [76]:
I11 = mp.quad(lambda t: t ** (-(k+1)/2 - 2 + 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-z * (t + r ** 2 / t)), [0, mp.inf])

In [77]:
I12 = mp.quad(lambda t: t ** (-(k+1)/2 - 2 + 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2))) * 1/z * z * (r**2/t**2 - 1) * mp.exp(-z * (t + r ** 2 / t)), [0, mp.inf])

In [88]:
(I11 + I12) / r ** 2

mpf('0.556845616943591680157993151047595')

In [89]:
I11, I12

(mpf('0.0223193521717060485394903919596669'),
 mpf('0.0124834988872684314703841799808063'))

In [90]:
mp.quad(lambda t: t ** (-k/2 + 3/2 - 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-z * (t + r ** 2 / t)), [0, mp.inf])

mpf('0.0223193521717060485394903919596669')